In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Parte 1
# Obtenção e carregamento do Dataset
# Criação dos objetos Dataset e pré-processamento dos dados

# Fazer um download de um dataset contendo imagens de caes e gatos.  
# O método get_file() faz o download e extrai a informação do zip: 
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
# A informação fica na pasta datasets da diretoria corrente (confirmar a divisão por pastas)

import os 

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True, cache_dir=os.curdir)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
# Criação e pré-processamento dos datasets (neste caso, vamos usar apenas datasets de treino e de validação)

IMG_SIZE = (160, 160)

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_ds = image_dataset_from_directory(train_dir, shuffle=True, seed=11, image_size=IMG_SIZE)
val_ds = image_dataset_from_directory(validation_dir, shuffle=True, seed=11, image_size=IMG_SIZE)

class_names = train_ds.class_names

train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)

normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
# Parte 2
# Análise detalhada das transformações aplicadas a imagens pelas camadas habituais de uma CNN

# Visualizar alguns exemplos

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("float32")) #pode ser transformado num array numpy 
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# Obter 2 imagens para análise mais detalhada 
# No código só estão a ser visualizados os efeitos na primeira imagem. Alterar para ver o que acontece com a outra

for images, labels in train_ds.take(1):
    im1 = images[0].numpy().astype("float32")
    label1 = labels[0]
    im2 = images[1].numpy().astype("float32")
    label2 = labels[1]
    
    
plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 2, 1)
plt.imshow(im1)
plt.title(class_names[label1])
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(im2)
plt.title(class_names[label2])
plt.axis("off")

In [ ]:
# Ajustar a dimensão do Tensor (adicionar a dimensão batch para que possa ser processada por layers)
im1 = tf.expand_dims(im1, axis=0)

im1.shape

In [ ]:
# Criar filtros verticais e horizontais 
# A última dimensão define o número de canais de output - neste caso é apenas 1 feature map por cada filtro

# Podem alterar os filtros para verificar se existem diferenças nos feature maps que são produzidos

filter1 = np.zeros(shape=(7,7,3,1), dtype = np.float32)

filter2 = np.zeros(shape=(7,7,3,1), dtype = np.float32)

filter1[: , 3 , : ,] = 2
filter1[: , 1 , : ,] = -1
filter1[: , 5 , : ,] = -1


filter2[3 , : , :,] = 2
filter2[1 , : , :,] = -1
filter2[5 , : , :,] = -1

In [ ]:
filter1.shape

In [ ]:
# Visualizar o filtro1 através das suas 3 camadas

a = filter1.squeeze()

R = a[:,:,0]
G = a[:,:,1]
B = a[:,:,2]

plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 3, 1)
plt.imshow(R)
plt.title("R")
plt.axis("off")
ax = plt.subplot(1, 3, 2)
plt.imshow(G)
plt.title("G")
plt.axis("off")
ax = plt.subplot(1, 3, 3)
plt.imshow(B)
plt.title("B")
plt.axis("off")

In [ ]:
# Completar o código para visualizar o filtro2

In [ ]:
# Aplicar CNN a cada uma das imagens
# Neste caso está a ser usado o método diretamente da biblioteca TensorFlow - Não estamos a usar a API Keras
# https://www.tensorflow.org/api_docs/python/tf/nn
# https://www.tensorflow.org/api_docs/python/tf/nn/conv2d
# Ainda não está a ser aplicada a função de ativação

output1 = tf.nn.conv2d(im1, filter1, strides=1, padding="SAME")
output2 = tf.nn.conv2d(im1, filter2, strides=1, padding="SAME")

In [ ]:
# Qual é a dimensão do output? Porquê?



In [ ]:
#Visualizar o feature map obtido por cada um dos filtros
#https://www.tensorflow.org/api_docs/python/tf/squeeze

fm1 = tf.squeeze(output1)
fm2 = tf.squeeze(output2)

plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(fm1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(fm2, cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")

# Analisar os padrões que se salientam nas imagens obtidas

In [ ]:
# Aplicar função de ativação RELU
# https://www.tensorflow.org/api_docs/python/tf/nn/relu

final1 = tf.nn.relu(output1)
final2 = tf.nn.relu(output2)

In [ ]:
# Confirmar a dimensão do output.



In [ ]:
#Visualizar o feature map obtido após a aplicação da função de ativação
#https://www.tensorflow.org/api_docs/python/tf/squeeze

fmF1 = tf.squeeze(final1)
fmF2 = tf.squeeze(final2)

plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(fmF1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(fmF2, cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")

In [ ]:
# Exercício: Qual a dimensão dos mapas de outputs em cada uma destas 4 situações?
# Dimensões de input: Imagem (160, 160, 3), Filtro (7, 7, 3)

outputE0 = tf.nn.conv2d(im1, filter1, strides=1, padding="SAME")

outputE1 = tf.nn.conv2d(im1, filter1, strides=2, padding="SAME")

outputE2 = tf.nn.conv2d(im1, filter1, strides=1, padding="VALID")

outputE3 = tf.nn.conv2d(im1, filter1, strides=2, padding="VALID")

In [ ]:
# Verificar os resultados aqui


In [ ]:
# Aplicação de uma camada MaxPool

pool1 = tf.nn.max_pool(final1, 2, strides=2, padding="SAME")
pool2 = tf.nn.max_pool(final2, 2, strides=2, padding="SAME")

In [ ]:
#Confirmar o Downsampling da imagem



In [ ]:
#Visualizar o feature map obtido após a aplicação da função de maxpool

plt.figure(figsize=(20, 20))

p1 = tf.squeeze(pool1)
p2 = tf.squeeze(pool2)

ax = plt.subplot(1, 2, 1)
plt.imshow(p1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(p2, cmap="binary")
plt.title("Horizontal", fontsize=30)
plt.axis("off")

In [ ]:
# Parte 3

# Criação, compilação e treino de uma CNN
# Análise de resultados 


# Criação de uma CNN Simples

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Vai ser usada a API funcional para criar o modelo
# https://www.tensorflow.org/api_docs/python/tf/keras/Model
#https://www.tensorflow.org/guide/keras/functional
# https://keras.io/api/layers/

inputs = keras.Input(shape=(160,160,3))
a = keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
b = keras.layers.MaxPooling2D()(a)
c = keras.layers.Conv2D(32, 3, activation='relu',padding='same')(b)
d = keras.layers.Conv2D(64, 3, activation='relu',padding='same')(c)
e = keras.layers.MaxPooling2D()(d)
f = keras.layers.Flatten()(e)
g = keras.layers.Dense(64, activation='relu')(f)
outputs = keras.layers.Dense(1, activation="sigmoid")(g)

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
# Exercício: Descrever as características desta CNN

# Quais são as camadas de extração de features?

# Quais são as camadas de classificação?


In [ ]:
# Mostrar detalhes e sumario
# Explicar o número de parâmetros de cada camada



In [ ]:
#Obter detalhes da terceira camada CNN (4ª camada da rede)

my_layer = model.layers[3]
my_layer.name

In [ ]:
weights, biases = my_layer.get_weights()
print("Weights Shape: ", weights.shape)
print("Biases Shape: ", biases.shape)

# Explicar os valores obtidos

In [ ]:
# Compilar o modelo 
# A função de loss está ajustada ao formato da saída

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
# Visualização da evolução da accurary e da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# Analisar resultados

In [ ]:
# Melhorar o desempenho através da alteração da arquitetura da CNN
# Experimente adicionar mecanismos de combate ao overfitting, como Dropout e/ou Batch Normalization
# Adicione um callback para Early Stopping


keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# CRIAR NOVO MODELO chamado model2 utilizando a API funcional



In [ ]:
model2.summary()

In [ ]:
# Compilar e treinar o novo modelo


In [ ]:
# Analisar resultados

